In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
matplotlib.rcParams['agg.path.chunksize'] = 10000


dataset = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')

In [ ]:
def clean_dataset(dataset):
    # make Age integers

    dataset.loc[dataset['Gender'] == 'Male', 'Gender'] = 1
    dataset.loc[dataset['Gender'] == 'Female', 'Gender'] = 0

    # make Vehicle Age integers

    dataset.loc[dataset['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 0
    dataset.loc[dataset['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 1
    dataset.loc[dataset['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2

    # make Vehicle Damage integers
    dataset.loc[dataset['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0
    dataset.loc[dataset['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
    
    return dataset

In [ ]:
dataset = clean_dataset(dataset)

In [ ]:
# inspect relationship between age and response 

temp_dataset = dataset.groupby('Age').mean()

plt.bar(temp_dataset.index, temp_dataset['Response'])
plt.xlabel('Age')
plt.ylabel('Response')
plt.title('Age vs Interest in Vehicle Insurance')

In [ ]:
# inspect relationship between gender and response 

temp_dataset = dataset.groupby('Gender').mean()

plt.bar(temp_dataset.index, temp_dataset['Response'])
plt.xlabel('Gender')
plt.ylabel('Response')
plt.title('Gender vs Interest in Vehicle Insurance')

In [ ]:
train_labels = dataset['Response']

del dataset['Response']

train_dataset = dataset

from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
train_dataset, train_labels = oversample.fit_resample(train_dataset, train_labels)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_dataset,train_labels)

In [ ]:
# prepare test dataset

test_dataset = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/test.csv')
test_dataset = clean_dataset(test_dataset)

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
output = pd.DataFrame({'id': np.arange(381110, 381110+len(predictions)), 
    'Response': predictions})

In [ ]:
output['Response'].value_counts()

In [ ]:
output.to_csv('predictions.csv', index=False)  